In [22]:
import nltk
import json
import pandas as pd

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Luka\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [23]:
model_path = "../../models"

In [24]:
data_path = '../../data/google_translate.jsonl'
with open(data_path, 'r') as f:
    data = [json.loads(line) for line in f]

In [25]:
def tree_to_pairs(tree):
    # get pairs for root prompt and replies
    pairs = message_to_pairs(tree["prompt"])
    return pairs

def message_to_pairs(message):
    pairs = []
    # check who is the input role (we want only pairs where the input is the prompter)
    input_text = message['translation']
    # check if we have any replies to build pairs from
    if 'replies' in message:
        for reply in message['replies']:
            if "translation" not in reply:
                continue
            # get pairs for current input text ang reply
            pairs.append((input_text, reply['translation'], reply['role']))
            # get pairs for reply and its replies
            pairs.extend(message_to_pairs(reply))
    return pairs

In [26]:
pairs = []
for tree in data:
    pairs += tree_to_pairs(tree)
pairs_df = pd.DataFrame(pairs, columns=['prompt', 'reply', 'reply_role'])

mask = pairs_df['reply_role'] == "assistant"
pairs_df = pairs_df[mask]
pairs_df.drop(columns=['reply_role'], inplace=True)
display(pairs_df)

,prompt,reply
0,Ali imate kakšne informacije o Commodore 64?,Seveda. Commodore 64 je 8-bitni domači računal...
1,Ali imate kakšne informacije o Commodore 64?,"Seveda, tukaj je kratek povzetek trenutnega čl..."
7,Ali imate kakšne informacije o Commodore 64?,Commodore 64 (C64) je 8-bitni domači računalni...
9,Ali imate kakšne informacije o Commodore 64?,ja Bi še kaj posebej radi vedeli?
16,Metoda DMAIC v akciji\nProduktna skupina opaža...,"Metoda DMAIC (Define, Measure, Analyze, Improv..."
...,...,...
67035,Katere jezike govorijo v Kostariki?,"Primarni jezik, ki se govori v Kostariki, je š..."
67036,Katere jezike govorijo v Kostariki?,Uradni nacionalni jezik Kostarike je španščina...
67037,Kaj je umetna nevronska mreža?,Umetna nevronska mreža (ANN) je računalniški m...
67038,Kaj je umetna nevronska mreža?,"Umetna nevronska mreža je računalniški model, ..."


### Something

In [27]:
from transformers import AutoModelForSeq2SeqLM, AutoModel, AutoTokenizer

from datasets import *
import numpy as np
import torch

SEED = 42

In [28]:
data = Dataset.from_pandas(pairs_df[["prompt", "reply"]]).train_test_split(test_size=0.3, shuffle=True, seed=SEED)
data

DatasetDict({
    train: Dataset({
        features: ['prompt', 'reply', '__index_level_0__'],
        num_rows: 33259
    })
    test: Dataset({
        features: ['prompt', 'reply', '__index_level_0__'],
        num_rows: 14255
    })
})

In [29]:
train_data, test_data = data["train"], data["test"]

In [30]:
metric = load_metric("rouge")

In [31]:
tokenizer_no_prefix = AutoTokenizer.from_pretrained(f"{model_path}/no-prefix-t5-sl-small-finetuned-assistant")
model_no_prefix = AutoModelForSeq2SeqLM.from_pretrained(f"{model_path}/no-prefix-t5-sl-small-finetuned-assistant").to("cuda")

inputs = test_data["prompt"][:10]
for input in inputs:
    print(input)
    input = tokenizer_no_prefix(input, return_tensors="pt").to("cuda")
    outputs = model_no_prefix.generate(**input, max_length=128, no_repeat_ngram_size=2, num_beams=5, num_return_sequences=5)
    tokenizer_no_prefix.decode(outputs[0], skip_special_tokens=True)
    temp = tokenizer_no_prefix.batch_decode(outputs, skip_special_tokens=True)
    for t in temp:
        print(t)

Super ! zdaj uporabite to funkcijo za seznam 100 naključnih int
Glejte, kako je to mogoče, če želite, da vam pomagam pri iskanju in iskanju informacij, ki jih lahko uporabite za iskanje in iskanje informacij. Če želite izvedeti, kaj se dogaja, se morate odločiti, ali je vaša funkcija pravilna ali napačna. To lahko storite, ko želite najti informacije o tem, kje se nahajate.
Glejte, kako je to mogoče, če želite, da vam pomagam pri iskanju in iskanju informacij, ki jih lahko uporabite za iskanje in iskanje informacij. Če želite izvedeti, kaj se dogaja, se morate odločiti, ali je vaša funkcija pravilna ali napačna.
Glejte, kako je to mogoče, če želite, da vam pomagam pri iskanju in iskanju informacij, ki jih lahko uporabite za iskanje in iskanje informacij. Če želite izvedeti, kaj se dogaja, se morate odločiti, ali je vaša funkcija pravilna ali ne.
Glejte, kako je to mogoče, če želite, da vam pomagam pri iskanju in iskanju informacij, ki jih lahko uporabite za iskanje in iskanje informaci

In [32]:
tokenizer_prefix_in = AutoTokenizer.from_pretrained(f"{model_path}/prefix-in-t5-sl-small-finetuned-assistant")
model_prefix_in = AutoModelForSeq2SeqLM.from_pretrained(f"{model_path}/prefix-in-t5-sl-small-finetuned-assistant").to("cuda")

inputs = test_data["prompt"][:10]
for input in inputs:
    print(input)
    input = tokenizer_no_prefix("Uporabnik: " + input, return_tensors="pt").to("cuda")
    outputs = model_no_prefix.generate(**input, max_length=128, no_repeat_ngram_size=2, num_beams=5, num_return_sequences=5)
    tokenizer_no_prefix.decode(outputs[0], skip_special_tokens=True)
    temp = tokenizer_no_prefix.batch_decode(outputs, skip_special_tokens=True)
    for t in temp:
        print(t)

Super ! zdaj uporabite to funkcijo za seznam 100 naključnih int
Glejte, kako je to mogoče, če želite, da vam pomagam pri iskanju in iskanju informacij, ki jih lahko uporabite za iskanje in iskanje informacij. Če želite izvedeti, kaj se vam zdi pomembno, vam lahko pomagate.
Glejte, kako je to mogoče, če želite, da vam pomagam pri iskanju in iskanju informacij, ki jih lahko uporabite za iskanje in iskanje informacij. Če želite izvedeti, kaj se vam zdi pomembno, vam lahko pomagate najti informacije o tem, kar želite.
Glejte, kako je to mogoče, če želite, da vam pomagam pri iskanju in iskanju informacij, ki jih lahko uporabite za iskanje in iskanje informacij. Če želite izvedeti, kaj se vam zdi pomembno, vam lahko pomagate najti informacije o tem, kar želite doseči.
Glejte, kako je to mogoče, če želite, da vam pomagam pri iskanju in iskanju informacij, ki jih lahko uporabite za iskanje in iskanje informacij. Če želite izvedeti, kaj se vam zdi pomembno, vam lahko pomagate najti informacije 

In [33]:
tokenizer_prefix = AutoTokenizer.from_pretrained(f"{model_path}/old-t5-sl-small-finetuned-assistant")
model_prefix = AutoModelForSeq2SeqLM.from_pretrained(f"{model_path}/old-t5-sl-small-finetuned-assistant").to("cuda")

inputs = test_data["prompt"][:10]
for input in inputs:
    print(input)
    input = tokenizer_no_prefix("Uporabnik: " + input, return_tensors="pt").to("cuda")
    outputs = model_no_prefix.generate(**input, max_length=128, no_repeat_ngram_size=2, num_beams=5, num_return_sequences=5)
    tokenizer_no_prefix.decode(outputs[0], skip_special_tokens=True)
    temp = tokenizer_no_prefix.batch_decode(outputs, skip_special_tokens=True)
    for t in temp:
        print(t)

Super ! zdaj uporabite to funkcijo za seznam 100 naključnih int
Glejte, kako je to mogoče, če želite, da vam pomagam pri iskanju in iskanju informacij, ki jih lahko uporabite za iskanje in iskanje informacij. Če želite izvedeti, kaj se vam zdi pomembno, vam lahko pomagate.
Glejte, kako je to mogoče, če želite, da vam pomagam pri iskanju in iskanju informacij, ki jih lahko uporabite za iskanje in iskanje informacij. Če želite izvedeti, kaj se vam zdi pomembno, vam lahko pomagate najti informacije o tem, kar želite.
Glejte, kako je to mogoče, če želite, da vam pomagam pri iskanju in iskanju informacij, ki jih lahko uporabite za iskanje in iskanje informacij. Če želite izvedeti, kaj se vam zdi pomembno, vam lahko pomagate najti informacije o tem, kar želite doseči.
Glejte, kako je to mogoče, če želite, da vam pomagam pri iskanju in iskanju informacij, ki jih lahko uporabite za iskanje in iskanje informacij. Če želite izvedeti, kaj se vam zdi pomembno, vam lahko pomagate najti informacije 